In [ ]:
import pandas as pd
train = pd.read_csv("../input/porto-seguro-safe-driver-prediction/train.csv")
#test = pd.read_csv("../input/porto-seguro-safe-driver-prediction/test.csv")
train.info()

In [ ]:
train.head()

In [ ]:
train.describe().T

In [ ]:
train.isnull().sum()

In [ ]:
def traitement_df(dataframe) :
    df = dataframe
    df = df.drop(['id'], axis=1)
    df.dropna(inplace=True,axis=0)
    return df

In [ ]:
train = traitement_df(train)
#test = traitement_df(test)

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(['target'], axis=1)
y = train.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
train.target.value_counts()

oulala déséquilibré

### Sous-échantillonage

In [ ]:
from imblearn.under_sampling import RandomUnderSampler 

rus = RandomUnderSampler()
X_train, y_train = rus.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
feature_names = [f"feature {i}" for i in range(X.shape[1])]


from sklearn import ensemble

rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

print(classification_report(y_test, y_rf))

cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

importances = rf.feature_importances_

feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest_importances = pd.Series(importances, index=feature_names)
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

pas ouf... (mais finalement l'un des moins pires)

### Suréchantillonnage

In [ ]:
from imblearn.over_sampling import SMOTE



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)## fit_sample/fit_resample (si il y a une erreur)

In [ ]:
y_train.value_counts()

In [ ]:
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

print(classification_report(y_test, y_rf))

cm = confusion_matrix(y_test, y_rf)
print(cm)

encore pire (Quasiment aucun cas minoritaire n'est détecté)

### Extreme Gradient Boosting : XGBoost avec suréchantillonage SMOTE

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
print(xgb.score(X_test,y_test))

In [ ]:
y_xgb = xgb.predict(X_test)

print(classification_report(y_test, y_xgb))

cm = confusion_matrix(y_test, y_xgb)
print(cm)

encore encore pire (Quasiment aucun cas minoritaire n'est détecté)

### XGBoost pondéré

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
y_train.value_counts()

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(scale_pos_weight=458809/17360)
# xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_xgb = xgb.predict(X_test)

print(classification_report(y_test, y_xgb))
cm = confusion_matrix(y_test, y_xgb)
print(cm)

pas trop mal, comparé au reste (résultats comparable au sous-échantillonage)

In [ ]:
!pip install mljar-supervised
from supervised.automl import AutoML 
from supervised.preprocessing.eda import EDA

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

EDA.extensive_eda(X_train,y_train,save_path=".")
automl = AutoML(mode="Explain", eval_metric="accuracy")
automl.fit(X_train, y_train)

automl.report()

https://www.kaggle.com/code/werooring/top-9th-lightgbm-xgboost-ensemble :
get dummies sur les colonnes "cat"

In [ ]:
train = pd.read_csv("../input/porto-seguro-safe-driver-prediction/train.csv")

In [ ]:
def traitement_df(dataframe) :
    df = dataframe
    df = df.drop(['id'], axis=1)
    df.dropna(inplace=True,axis=0)
    
    all_features = df.columns.tolist() # All features
    cat_features = [col for col in all_features if 'cat' in col] # Nominal features
    df = pd.get_dummies(df,columns = cat_features)
    return df

In [ ]:
train = traitement_df(train)
train.head()

In [ ]:
X = train.drop(['target'], axis=1)
y = train.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

EDA.extensive_eda(X_train,y_train,save_path=".")
autom2 = AutoML(mode="Explain", eval_metric="accuracy")
autom2.fit(X_train, y_train)

autom2.report()

In [ ]:
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv')
test = traitement_df(test)

In [ ]:
autom1.predict(test)